# Run RC

In [1]:
import os
import subprocess
import csv
import pandas as pd
from joblib import Parallel, delayed

In [2]:
signals = ['fBrownTurb_H075_D1',
'LorenzX_s50_patch_IkedaX',
'RosslerX_s20',
'HenonX',              
'LorenzX_s50',   
'White',
'IkedaX',             
'MackeyGlass_t17']

SNRs = [0, 5, 10, 25, 50, 100]

networks = os.listdir('./Networks/Emp_connectomes')

## Empirical connectomes

In [ ]:
net_folder = './Networks/Emp_connectomes'
sig_folder = './signals'
res_folder = './results_Emp'

In [ ]:
for j in signals:
    if not os.path.exists(res_folder + '/'+ j):
        os.mkdir(res_folder + '/'+ j)

In [ ]:
SNRs[0:1]

In [ ]:
def run_emp(networks, signal, SNRs):
    
    for network in networks[4:5]: #networks:
        for SNR in SNRs[0:1]:  #SNRs:
            
            res_name = res_folder + '/' + signal + '/' + network + '__' + signal + '.txt' + '__' + str(SNR) + '.rds'

            if not os.path.exists(res_name):
                subprocess.run(['Rscript', 'batch_analysis.R',
                                            '0', # <N>
                                            net_folder + '/' + network, #<edgeslist>
                                            sig_folder + '/' + signal + '.txt', # <signal>
                                            str(SNR), # <SNR>
                                            '20', #<MC> realizzazioni random di RC?
                                            res_name # <outfile.rds>
                                           ])
                
# subprocess running external programs and reading their outputs in your Python code.

In [ ]:
Parallel(n_jobs=-1, verbose=10)(delayed(run_emp)(networks, signal, SNRs) for signal in signals)

## Synthetic connectomes

In [ ]:
def run_syn(networks, signal, SNRs):

    for r in range(1): #range(20): # 20 different random initializations of the weights

        if not os.path.exists(res_folder + '/' + signal + '/' + str(r)):
            os.mkdir(res_folder + '/' + signal + '/' + str(r))

        for network in networks[4:5]: #networks:
            for SNR in SNRs[0:1]:  #SNRs:

                res_name = res_folder + '/' + signal + '/' + str(r) + '/'+ network + '__' + signal + '.txt' + '__' + str(SNR) + '.rds'
                
                if not os.path.exists(res_name):
                    subprocess.run(['Rscript', 'batch_analysis.R',
                                    '0', 
                                    net_folder + '/' + str(r) + '/' + network,
                                    sig_folder + '/' + signal + '.txt', 
                                    str(SNR), 
                                    '1',
                                    res_name
                                   ])

### Configuration Models

In [ ]:
net_folder = './Networks/CM_connectomes'
sig_folder = './signals'
res_folder = './results_CM'

In [ ]:
for j in signals:
    if not os.path.exists(res_folder + '/'+ j):
        os.mkdir(res_folder + '/'+ j)

In [ ]:
Parallel(n_jobs=-1, verbose=10)(delayed(run_syn)(networks, signal, SNRs) for signal in signals)

### Erdos Renyi

In [ ]:
net_folder = './Networks/ER_connectomes'
sig_folder = './signals'
res_folder = './results_ER'

In [ ]:
for j in signals:
    if not os.path.exists(res_folder + '/'+ j):
        os.mkdir(res_folder + '/'+ j)

In [ ]:
Parallel(n_jobs=-1, verbose=10)(delayed(run_syn)(networks, signal, SNRs) for signal in signals)

# Process results

In [ ]:
def getMSE(Y, Yt, errorLen, norm = False):
           
    mse = np.sum((Y[:errorLen] - Yt[:errorLen]) ** 2) / errorLen
        
    if norm == True:
        mse = mse / np.std(Yt[:errorLen])
        
    return mse  

## Empirical connectomes

In [ ]:
def MSE_writer_emp(file_name, path, signal, res_name, r):

    file_path = path + '/' + signal + '/' + str(r) + '/' + file_name          
    file_rds = robjects.r['readRDS'](file_path)
    
    gc.collect()
                
    base, ext = os.path.splitext(file_name)
    base, ext = base.rsplit("__", 1)
    base, gar = os.path.splitext(base)
    base, gar = base.rsplit("__", 1)
    
    tau = [2**(i) for i in range(2,13)]
    initLen = 100
    trainLen = 2500
    testLen = 5000

    Yt = np.array(file_rds[0][0][trainLen+1:trainLen+testLen+1]).reshape(testLen,1)
    Y = np.array(file_rds[0][1]).reshape(testLen,1)

    for t in tau:
        mse = getMSE(Y, Yt, t, False)
        mse_norm = getMSE(Y, Yt, t, True)

        with open(res_name, "a") as f:
            writer = csv.writer(f)
            writer.writerow((base, ext, r, t, mse, mse_norm))
    
    del file_rds
    gc.collect()

## Synthetic connectomes

In [ ]:
def MSE_writer_syn(file_name, path, signal, res_name, r):

    file_path = path + '/' + signal + '/' + str(r) + '/' + file_name          
    file_rds = robjects.r['readRDS'](file_path)
    
    gc.collect()
                
    base, ext = os.path.splitext(file_name)
    base, ext = base.rsplit("__", 1)
    base, gar = os.path.splitext(base)
    base, gar = base.rsplit("__", 1)
    
    tau = [2**(i) for i in range(2,13)]
    initLen = 100
    trainLen = 2500
    testLen = 5000

    Yt = np.array(file_rds[0][0][trainLen+1:trainLen+testLen+1]).reshape(testLen,1)
    Y = np.array(file_rds[0][1]).reshape(testLen,1)

    for t in tau:
        mse = getMSE(Y, Yt, t, False)
        mse_norm = getMSE(Y, Yt, t, True)

        with open(res_name, "a") as f:
            writer = csv.writer(f)
            writer.writerow((base, ext, r, t, mse, mse_norm))
    
    del file_rds
    gc.collect()

In [ ]:
def process_signal_syn(signal, res_folder):
    
    res_name = './' + res_folder + '/' + signal + '.txt'

    if os.path.exists(res_name):
        os.remove(res_name)
    
    for r in range(20):
        file_names = os.listdir(path + '/' + signal + '/' + str(r))
        file_names.sort()

        for i in range(len(file_names)):
            print(signal, r, i)
            MSE_writer(file_names[i], path, signal, res_name, r)

### Configuration Model

In [ ]:
res_folder = 'results_CM_processed'
path = './results_CM'

if not os.path.exists(res_folder):
    os.mkdir(res_folder)

In [ ]:
Parallel(n_jobs=-1, verbose=50)(delayed(process_signal_syn)(signal, res_folder) for signal in signals)

### Erdos Renyi

In [ ]:
res_folder = 'results_ER_processed'
path = './results_ER'

if not os.path.exists(res_folder):
    os.mkdir(res_folder)

In [ ]:
Parallel(n_jobs=-1, verbose=50)(delayed(process_signal_syn)(signal, res_folder) for signal in signals)

In [4]:
signal=signals[0]
path='/Volumes/GoogleDrive/Il mio Drive/backup_Michele/results_SBM'
for r in range(20):
    file_names = os.listdir(path + '/' + signal + '/' + str(r))
    file_names.sort()

    for i in range(len(file_names)):
        print(signal, r, i)
        #MSE_writer(file_names[i], path, signal, res_name, r)
        break
    break

fBrownTurb_H075_D1 0 0


### Rank covariance matrix